### Ollama

In [2]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED     
nomic-embed-text:latest	0a109f422b47	274 MB	11 days ago 	
llama3:8b              	365c0bd3c000	4.7 GB	2 weeks ago 	
llama3.1:8b            	75382d0899df	4.7 GB	3 weeks ago 	
llama3:text            	870a5d02cfaf	4.7 GB	3 weeks ago 	
qwen2:7b               	e0d4e1163c58	4.4 GB	3 weeks ago 	
gemma2:9b              	ff02c3702f32	5.4 GB	3 weeks ago 	
mistral:latest         	2ae6f6dd7a3d	4.1 GB	2 months ago	


In [3]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [4]:
local_model = "gemma2:9b"
llm = ChatOllama(model=local_model)

In [5]:
llm

ChatOllama(model='gemma2:9b')

In [6]:
import pandas as pd

df = pd.read_json(
    r'F:\AI\Super AI SS4\Level 3 - INTERN\Dataset\THEconSum\AIFORTHAI-TextSummarizationCorpus\Text-Summarization-Corpus.json')

df.shape

(3189, 5)

In [6]:
def summarize_abstractive(text, llm):
    prompt = f"""Write a summary in a narrative style that presents the content in a continuous, flowing manner, focusing on a chronological order of events. The narrative should be smooth without abrupt breaks, covering all relevant points comprehensively, but without delving into deep, specific details. The content should provide a clear overview of the entire event or story to give readers a broad understanding.
    Please read the following news article carefully and follow the instructions below

    {text}

    Instructions:
    - Write each paragraph based on the sequence of events or background, focusing on a continuous storytelling approach.
    - Use \t at the beginning of each paragraph to create indentation.
    - DO NOT leave blank lines between paragraphs. All paragraphs must be continuous with no blank lines.
    - There is NO need to give titles for each paragraph. Focus on telling the story while covering key points.

    IMPORTANT: All your answers must be in Thai. Ensure that the Thai translation accurately conveys the meaning and context of the original content."""

    system_template = f"""You are an expert summarizer with the ability to distill complex information into concise and accurate summaries. Your summaries are clear, coherent, and excellent at identifying key points while preserving the essence of the original text."""

    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_message_prompt = HumanMessagePromptTemplate.from_template(prompt)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    messages = chat_prompt.format_messages(text=text)

    response = llm(messages)
    return response.content

In [7]:
def summarize_extractive(text, llm):
    prompt = f"""Create a summary by selecting and combining key sentences directly from the original text. The summary should present the content in a chronological order, covering all relevant points comprehensively, but without adding any new words or phrases not present in the original text. The extracted sentences should provide a clear overview of the entire event or story to give readers a broad understanding.
    Please read the following news article carefully and follow the instructions below:

    {text}

    Instructions:
    - Use \t at the beginning of each paragraph to create indentation.
    - DO NOT leave blank lines between paragraphs. All paragraphs must be continuous with no blank lines.
    - DO NOT add any new words or phrases. Use ONLY the exact text from the original article.
    - There is NO need to give titles for each paragraph. Focus on presenting the key information while maintaining the original wording.
    - Please DO NOT generate anything else, generate ONLY the summary.

    IMPORTANT: Ensure that you only extract and combine existing sentences from the Thai text. Do not translate or rephrase any content."""

    system_template = f"""You are an expert in extractive summarization with the ability to identify and select the most important sentences from complex texts. Your summaries are coherent compilations of original sentences that accurately represent the key points and overall narrative of the source material."""

    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_message_prompt = HumanMessagePromptTemplate.from_template(prompt)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    messages = chat_prompt.format_messages(text=text)

    response = llm(messages)
    return response.content

In [8]:
df_sum = df[['content', 'extractive', 'abstractive']].sample(n=10, random_state=1).reset_index(drop=True)

df_sum.to_csv(
    r'F:\AI\Super AI SS4\Level 3 - INTERN\Dataset\ThEconSum\AIFORTHAI-TextSummarizationCorpus\ThEconSum-random-10-10.csv', index=False)

In [9]:
df_sum

,content,extractive,abstractive
0,กรุงเทพธุรกิจ หุ้นกลุ่มปตท.พลิกกับมา...,\t หุ้นกลุ่มปตท.พลิกกลับมานำ ตลาดหุ้นไทยฟื้น...,\t หุ้นกลุ่มปตท.พลิกกลับมานำ ตลาดหุ้นไทยฟื้นตั...
1,โครงการเมกะเวิลด์ รถไฟความเร็วสูงเชื...,\t โครงการเมกะเวิลด์ รถไฟความเร็วสูงเชื่อม 3 ส...,\t โครงการเมกะเวิลด์ รถไฟความเร็วสูงเชื่อม 3 ส...
2,กรุงเทพธุรกิจ สมคิด เรียกประชุมอธิกา...,\tวันที่ 28 ต.ค. นายสมคิด จาตุศรีพิทักษ์ รอง น...,วันที่ 28 ต.ค. นายสมคิด จาตุศรีพิทักษ์ รอง นาย...
3,การต่อสู้เรียกร้องให้ ปตท.คืนทรัพย์ส...,\tการต่อสู้เรียกร้องให้ ปตท.คืนทรัพย์สิน...,\tการต่อสู้เรียกร้องของภาคประชาชนโดยมูลน...
4,ศิริวรรณ ศรีเอี่ยมจันทร์ ใน...,\tช่วง 2-3 ปีที่ผ่านมา ราคาน้ำมันดิบในตลาดโลก...,\tช่วง 2-3 ปีที่ผ่านมา ราคาน้ำมันดิบในตลาดโลก...
5,* ดัชนีตลาดหลักทรัพย์ไทยวานนี้ (9 ต....,\tดัชนีตลาดหลักทรัพย์ แกว่งทั้งวัน รอผลเจรจากา...,\tดัชนีตลาดหลักทรัพย์ 10 ต.ค. แกว่งทั้งวัน ปิด...
6,บิ๊ก OCEAN เดินหน้าซื้อขายปาล์มน้ำมั...,นางชัชชญา ไตรตระกูลชัย กรรมการผู้จัด...,นางชัชชญา ไตรตระกูลชัย กรรมการผู้จัด...
7,เอราวัณ 3 ปีแห่งการครองอำนา...,\t3 ปีแห่งการครองอำนาจของคณะรักษาความสงบแห่งชา...,\tตลอด 3 ปี ของคณะรักษาความสงบแห่งชาติ (คสช.) ...
8,สุดใจ ชาญชาตรีรัตน์ ในอดีต ...,\tทิศทางราคาน้ำมันขาลงช่วง 3 ปีที่ผ่านมา ด้ว...,\tทิศทางราคาน้ำมันขาลงช่วง 3 ปีที่ผ่านมา ด้วยป...
9,“ปตท.” ออกโรงเอง! ลุยเจรจาผู้ผลิตก๊า...,นายอธิคม เติบศิริ ประธานเจ้าหน้าที่ปฏิบัติการก...,นายอธิคม เติบศิริ ประธานเจ้าหน้าที่ปฏิบัติการก...


In [10]:
text = df_sum.iloc[0]['content']
print(text)

          กรุงเทพธุรกิจ หุ้นกลุ่มปตท.พลิกกับมานำ ตลาดหุ้นไทยฟื้นตัว หนุนจากราคาน้ำมันดิบพุ่งขึ้น 10% หลังบริษัทน้ำมันในสหรัฐยื่นขอล้มละลาย โบรกประเมิน ปตท.สผ. อยู่รอดหลังต้นทุนเงินสดยังต่ำกว่าราคาปัจจุบัน ขณะที่หุ้น โออาร์ เดินหน้าขายไอพีโอ 2,700 ล้านหุ้น          ความเคลื่อนไหวหุ้นในกลุ่มปตท. วานนี้ (2 เม.ย.)  ได้แก่ บมจ.ปตท(PTT) บมจ.ปตท.สำรวจและผลิตปิโตรเลียม(PTTEP) บมจ.ไทยออยล์(TOP) และ บมจ.พีทีที โกลบอล เคมิคอล(PTTGC) ต่างปรับตัวขึ้น ได้กว่า 10% และเป็นกลุ่มที่ช่วยหนุนให้ดัชนีปรับตัวขึ้น ได้ราว 17 จุด จากทั้งหมด 32 จุด มาปิดที่ 1,138.27 จุด เพิ่มขึ้น 2.96% ด้วยมูลค่าการซื้อขาย 6.9 หมื่นล้านบาท          การปรับตัวขึ้นของหุ้นในกลุ่มปตท. ได้แรงหนุนสำคัญจากราคาน้ำมันดิบ (อิง WTI) ปรับตัวขึ้น 10% ตั้งแต่ช่วงเช้า กลับมายืนเหนือระดับ 22 ดอลลาร์ ต่อบาร์เรล โดยปัจจัยที่เกี่ยวข้องล่าสุด คือ บริษัท Whiting Petroleum ซึ่งเป็นหนึ่งในบริษัทเชลออยล์ของสหรัฐได้ประกาศล้มละลาย เนื่องจากราคาขายไม่คุ้มค่าในการผลิต และมีแนวโน้มที่อีกหลายบริษัทจะเผชิญสภาวะเดียวกัน เป็นผลให้ ประธานาธิบดี โดนัลด์ ทรัมป์ ข

In [ ]:
summarize_abstractive(text, llm)

In [18]:
df_sum = pd.read_csv(
    r'F:\AI\Super AI SS4\Level 3 - INTERN\Jupyter Notebook\Evaluate\gemma2-ThEconSum-20.csv')

df_sum.shape

(20, 6)

In [11]:
from tqdm import tqdm

In [12]:
tqdm.pandas()

df_sum['sum_abstractive'] = df_sum['content'].progress_apply(
    lambda x: summarize_abstractive(x, llm)
)

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\Nitro 5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
100%|██████████| 10/10 [43:05<00:00, 258.59s/it]


In [20]:
tqdm.pandas()

df_sum['sum_extractive'] = df_sum['content'].progress_apply(
    lambda x: summarize_extractive(x, llm)
)

100%|██████████| 20/20 [1:47:05<00:00, 321.28s/it]


In [21]:
df_sum.to_csv(
    r'F:\AI\Super AI SS4\Level 3 - INTERN\Jupyter Notebook\Evaluate\gemma2-ThEconSum-20.csv')
df_sum.shape

(20, 7)

### combine-dataset

In [28]:
df_sum = pd.read_csv(
    r'F:\AI\Super AI SS4\Level 3 - INTERN\Jupyter Notebook\Evaluate\gemma2-ThEconSum-20.csv')

df_sum.shape

(20, 5)

In [29]:
df_copy = df_sum[['content', 'abstractive', 'sum_abstractive']].copy()
df_copy.columns

Index(['content', 'abstractive', 'sum_abstractive'], dtype='object')

In [16]:
from bert_score import score
from pythainlp.tokenize import word_tokenize
from rouge import Rouge
from tqdm import tqdm

def calculate_bert_score(candidate, reference):
    P, R, F1 = score([candidate], [reference], lang='th', verbose=False)
    bert_score = {
        'precision': float(P[0]),
        'recall': float(R[0]),
        'f1': float(F1[0])
    }
    return bert_score

### BERT Score

In [57]:
bert_precisions = []
bert_recalls = []
bert_f1_scores = []

for index, row in tqdm(df_copy.iterrows(), total=df_copy.shape[0], desc="BERT Scores ..."):
    candidate = row['sum_extractive']
    reference = row['extractive']

    bert_score = calculate_bert_score(candidate, reference)

    bert_precisions.append(bert_score['precision'])
    bert_recalls.append(bert_score['recall'])
    bert_f1_scores.append(bert_score['f1'])

df_copy['BERT_Precision'] = bert_precisions
df_copy['BERT_Recall'] = bert_recalls
df_copy['BERT_F1'] = bert_f1_scores

BERT Scores ...: 100%|██████████| 20/20 [01:43<00:00,  5.19s/it]


### ROUGE Score

In [30]:
def calculate_rouge_scores_for_row(candidate, reference, engine=None):
    if engine:
        hypothesis_tokenized = ' '.join(word_tokenize(candidate, engine=engine))
        reference_tokenized = ' '.join(word_tokenize(reference, engine=engine))
    else:  # ใช้ default
        hypothesis_tokenized = ' '.join(word_tokenize(candidate))
        reference_tokenized = ' '.join(word_tokenize(reference))

    rouge = Rouge()
    scores = rouge.get_scores(hypothesis_tokenized, reference_tokenized)[0]

    rouge1_f1 = scores['rouge-1']['f']
    rouge2_f1 = scores['rouge-2']['f']
    rougel_f1 = scores['rouge-l']['f']

    avg_rouge_score = (rouge1_f1 + rouge2_f1 + rougel_f1) / 3

    return rouge1_f1, rouge2_f1, rougel_f1, avg_rouge_score

def calculate_update_dataframe_ab(df, engine=None):
    rouge1_scores = []
    rouge2_scores = []
    rougel_scores = []
    rouge_avg_scores = []

    engine_label = engine if engine else 'default'

    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Calculating ROUGE Scores with {engine_label}"):
        candidate = row['sum_abstractive']
        reference = row['abstractive']

        rouge1_f1, rouge2_f1, rougel_f1, avg_rouge_score = calculate_rouge_scores_for_row(candidate, reference, engine=engine)

        rouge1_scores.append(rouge1_f1)
        rouge2_scores.append(rouge2_f1)
        rougel_scores.append(rougel_f1)
        rouge_avg_scores.append(avg_rouge_score)

    df[f'ROUGE-1_{engine_label}'] = rouge1_scores
    df[f'ROUGE-2_{engine_label}'] = rouge2_scores
    df[f'ROUGE-L_{engine_label}'] = rougel_scores
    df[f'ROUGE_Score_{engine_label}'] = rouge_avg_scores

    return df

def calculate_update_dataframe_ex(df, engine=None):
    rouge1_scores = []
    rouge2_scores = []
    rougel_scores = []
    rouge_avg_scores = []

    engine_label = engine if engine else 'default'

    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Calculating ROUGE Scores with {engine_label}"):
        candidate = row['sum_extractive']
        reference = row['extractive']

        rouge1_f1, rouge2_f1, rougel_f1, avg_rouge_score = calculate_rouge_scores_for_row(candidate, reference, engine=engine)

        rouge1_scores.append(rouge1_f1)
        rouge2_scores.append(rouge2_f1)
        rougel_scores.append(rougel_f1)
        rouge_avg_scores.append(avg_rouge_score)

    df[f'ROUGE-1_{engine_label}'] = rouge1_scores
    df[f'ROUGE-2_{engine_label}'] = rouge2_scores
    df[f'ROUGE-L_{engine_label}'] = rougel_scores
    df[f'ROUGE_Score_{engine_label}'] = rouge_avg_scores

    return df

### Abstractive

In [21]:
df_copy_attacut = df_copy.copy()
df_copy_default = df_copy.copy()

df_copy_attacut = calculate_update_dataframe_ab(df_copy_attacut, engine='attacut')
df_copy_default = calculate_update_dataframe_ab(df_copy_default)  # ใช้ default

avg_attacut = df_copy_attacut[[f'ROUGE-1_attacut', f'ROUGE-2_attacut', f'ROUGE-L_attacut', f'ROUGE_Score_attacut']].mean()
avg_default = df_copy_default[[f'ROUGE-1_default', f'ROUGE-2_default', f'ROUGE-L_default', f'ROUGE_Score_default']].mean()

comparison_df = pd.DataFrame({
    'Metric': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE_Score'],
    'Average_Attacut': avg_attacut.values,
    'Average_Default': avg_default.values
})

Calculating ROUGE Scores with default: 100%|██████████| 20/20 [00:00<00:00, 23.42it/s]


In [23]:
comparison_df

,Metric,Average_Attacut,Average_Default
0,ROUGE-1,0.642744,0.600885
1,ROUGE-2,0.384514,0.371114
2,ROUGE-L,0.595207,0.560466
3,ROUGE_Score,0.540822,0.510822


In [31]:
df_copy = df_sum[['content', 'extractive', 'sum_extractive']].copy()
df_copy.columns

Index(['content', 'extractive', 'sum_extractive'], dtype='object')

### Extractive

In [32]:
df_copy_attacut = df_copy.copy()
df_copy_default = df_copy.copy()

df_copy_attacut = calculate_update_dataframe_ex(df_copy_attacut, engine='attacut')
df_copy_default = calculate_update_dataframe_ex(df_copy_default)  # ใช้ default

avg_attacut = df_copy_attacut[[f'ROUGE-1_attacut', f'ROUGE-2_attacut', f'ROUGE-L_attacut', f'ROUGE_Score_attacut']].mean()
avg_default = df_copy_default[[f'ROUGE-1_default', f'ROUGE-2_default', f'ROUGE-L_default', f'ROUGE_Score_default']].mean()

comparison_df = pd.DataFrame({
    'Metric': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE_Score'],
    'Average_Attacut': avg_attacut.values,
    'Average_Default': avg_default.values
})

Calculating ROUGE Scores with default: 100%|██████████| 20/20 [00:00<00:00, 21.08it/s]


In [33]:
comparison_df

,Metric,Average_Attacut,Average_Default
0,ROUGE-1,0.678338,0.646395
1,ROUGE-2,0.478567,0.465378
2,ROUGE-L,0.657657,0.626743
3,ROUGE_Score,0.604854,0.579505
